In [1]:
!pip install tensorflow-addons
!pip install -q -U tensorflow-text
!pip install -q -U tf-models-official
!pip install -U tfds-nightly

     |████████████████████████████████| 1.1 MB 5.3 MB/s 
     |████████████████████████████████| 4.9 MB 5.3 MB/s 
     |████████████████████████████████| 1.8 MB 5.4 MB/s 
     |████████████████████████████████| 213 kB 58.4 MB/s 
     |████████████████████████████████| 99 kB 10.1 MB/s 
     |████████████████████████████████| 90 kB 2.6 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 47.3 MB/s 
     |████████████████████████████████| 47.7 MB 1.3 MB/s 
     |████████████████████████████████| 352 kB 71.8 MB/s 
     |████████████████████████████████| 596 kB 68.1 MB/s 
     |████████████████████████████████| 4.1 MB 5.4 MB/s 


In [2]:
!pip install xin-util -U

     |████████████████████████████████| 398 kB 5.2 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# """
# !pip install tensorflow-addons
# !pip install -q -U tensorflow-text
# !pip install -q -U tf-models-official
# !pip install -U tfds-nightly

# """

# import os
# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text  # A dependency of the preprocessing model
# import tensorflow_addons as tfa # For metrics
# from official.nlp import optimization

# tf.get_logger().setLevel('ERROR')

# """
# Configure TFHub to read checkpoints directly from TFHub's Cloud Storage buckets. This is only recommended when running TFHub models on TPU.
# Without this setting TFHub would download the compressed file and extract the checkpoint locally. Attempting to load from these local files will fail with the following error:
# ```
# InvalidArgumentError: Unimplemented: File system scheme '[local]' not implemented
# ```
# This is because the [TPU can only read directly from Cloud Storage buckets](https://cloud.google.com/tpu/docs/troubleshooting#cannot_use_local_filesystem).
# Note: This setting is automatic in Colab.
# """
# os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"


# class BERT_FineTune:
#     def __init__(self, handle_encoder, handle_preprocess):
#       try:
#         resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#         tf.config.experimental_connect_to_cluster(resolver)
#         tf.tpu.experimental.initialize_tpu_system(resolver)
#         strategy = tf.distribute.TPUStrategy(resolver)
#         print('Using TPU')
#       except:
#         if tf.config.list_physical_devices('GPU'):
#             strategy = tf.distribute.MirroredStrategy()
#             print('Using GPU')
#         else:
#             strategy = None
#             print('Running on CPU is not recommended.')
#       self.strategy = strategy
#       self.handle_encoder = handle_encoder
#       self.handle_preprocess = handle_preprocess

#     @staticmethod
#     def make_bert_preprocess_model(handle_preprocess, sentence_features, seq_length=128):
#         """Returns Model mapping string features to BERT inputs.

#         Args:
#         sentence_features: a list with the names of string-valued features.
#         seq_length: an integer that defines the sequence length of BERT inputs.

#         Returns:
#         A Keras Model that can be called on a list or dict of string Tensors
#         (with the order or names, resp., given by sentence_features) and
#         returns a dict of tensors for input to BERT.
#         """

#         input_segments = [
#             tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
#             for ft in sentence_features]

#         # Tokenize the text to word pieces.
#         bert_preprocess = hub.load(handle_preprocess)
#         tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
#         segments = [tokenizer(s) for s in input_segments]

#         # Optional: Trim segments in a smart way to fit seq_length.
#         # Simple cases (like this example) can skip this step and let
#         # the next step apply a default truncation to approximately equal lengths.
#         truncated_segments = segments

#         # Pack inputs. The details (start/end token ids, dict of output tensors)
#         # are model-dependent, so this gets loaded from the SavedModel.
#         packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
#                                 arguments=dict(seq_length=seq_length),
#                                 name='packer')
#         model_inputs = packer(truncated_segments)
#         return tf.keras.Model(input_segments, model_inputs)

#     @staticmethod
#     def convert_dataset(df, batch_size, bert_preprocess_model, sentence_features, label='label', shuffle=False, repeat=False):
#         AUTOTUNE = tf.data.AUTOTUNE
#         in_memory_ds = dict()
#         for feature in sentence_features:
#             in_memory_ds[feature] = df[feature]
#         in_memory_ds['label'] = df[label]

#         dataset = tf.data.Dataset.from_tensor_slices(in_memory_ds)
#         num_examples = len(in_memory_ds['label'])

#         if shuffle:
#             dataset = dataset.shuffle(num_examples)
#         if repeat:
#             dataset = dataset.repeat()
#         dataset = dataset.batch(batch_size)
#         dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex['label']))
#         dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
#         return dataset, num_examples

#     @staticmethod
#     def build_classifier_model(encoder, num_classes, seed=0):
#         tf.random.set_seed(seed)

#         class Classifier(tf.keras.Model):
#             def __init__(self, encoder, num_classes):
#                 super(Classifier, self).__init__(name="prediction")
#                 self.encoder = encoder
#                 self.dropout = tf.keras.layers.Dropout(0.1)
#                 self.dense = tf.keras.layers.Dense(num_classes)

#             def call(self, preprocessed_text):
#                 encoder_outputs = self.encoder(preprocessed_text)
#                 pooled_output = encoder_outputs["pooled_output"]
#                 x = self.dropout(pooled_output)
#                 x = self.dense(x)
#                 return x

#         model = Classifier(encoder, num_classes)
#         return model


#     def tune(self, train_df, val_df=None, sentence_features=['sentence'], label='label', epochs=3, batch_size=32, optimizer='sgd', seq_length=128):
#         num_classes = len(train_df[label].unique())
#         bert_preprocess_model = BERT_FineTune.make_bert_preprocess_model(self.handle_preprocess, sentence_features, seq_length)

#         train_dataset, train_data_size = BERT_FineTune.convert_dataset(
#             train_df, batch_size, bert_preprocess_model, sentence_features, label, True, True)

#         steps_per_epoch = train_data_size // batch_size
#         num_train_steps = steps_per_epoch * epochs
#         num_warmup_steps = num_train_steps // 10
        
#         if val_df is None:
#             validation_dataset = None
#             validation_steps = None
#         else:
#             validation_dataset, validation_data_size = BERT_FineTune.convert_dataset(
#                 val_df, batch_size, bert_preprocess_model, sentence_features, label, False, False)
#             validation_steps = validation_data_size // batch_size

        
#         if self.strategy is None:
#             encoder = hub.KerasLayer(self.handle_encoder, trainable=True)

#             # metric have to be created inside the strategy scope

#             loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#             #metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=2)
#             classifier_model = BERT_FineTune.build_classifier_model(encoder, num_classes)

#             # optimizer = optimization.create_optimizer(
#             #     init_lr=init_lr,
#             #     num_train_steps=num_train_steps,
#             #     num_warmup_steps=num_warmup_steps,
#             #     optimizer_type='adamw')

#             classifier_model.compile(optimizer=optimizer, loss=loss, metrics='accuracy') # metrics=[metrics]

#             classifier_model.fit(
#                 x=train_dataset,
#                 validation_data=validation_dataset,
#                 steps_per_epoch=steps_per_epoch,
#                 epochs=epochs,
#                 validation_steps=validation_steps)
#         else:
#             with self.strategy.scope():
#                 encoder = hub.KerasLayer(self.handle_encoder, trainable=True)

#                 # metric have to be created inside the strategy scope

#                 loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#                 #metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=2)

#                 classifier_model = BERT_FineTune.build_classifier_model(encoder, num_classes)

#                 # optimizer = optimization.create_optimizer(
#                 #     init_lr=init_lr,
#                 #     num_train_steps=num_train_steps,
#                 #     num_warmup_steps=num_warmup_steps,
#                 #     optimizer_type='adamw')

#                 classifier_model.compile(optimizer=optimizer, loss=loss, metrics='accuracy') # metrics=[metrics]

#                 classifier_model.fit(
#                     x=train_dataset,
#                     validation_data=validation_dataset,
#                     steps_per_epoch=steps_per_epoch,
#                     epochs=epochs,
#                     validation_steps=validation_steps)
#         return classifier_model



In [5]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/drive/MyDrive/Data/IMDB Dataset.csv')
data=data.head(100)
#data = pd.read_csv('/content/drive/MyDrive/Data/positive_data.csv')

In [6]:
label_map = {'positive': 1, 'negative': 0}
data.sentiment.replace(label_map, inplace=True)


In [7]:
msk = np.random.rand(len(data)) < 0.8

train_df = data[msk]

val_df = data[~msk]

In [8]:
val_df

,review,sentiment
1,A wonderful little production. <br /><br />The...,1
12,So im not a big fan of Boll's work but then ag...,0
15,"Kind of drawn in by the erotic scenes, only to...",0
17,This movie made it into one of my top 10 most ...,0
26,"""The Cell"" is an exotic masterpiece, a dizzyin...",1
27,This film tried to be too many things all at o...,0
61,"I've just watched Fingersmith, and I'm stunned...",0
69,This film laboured along with some of the most...,0
77,"Maybe it was the title, or the trailer (certai...",0
82,"Oh God, I must have seen this when I was only ...",0


In [9]:
from ModelHub.BERT import BERT_FineTune

In [10]:
feature_cols = ['review']

In [11]:
# bert_finetune = BERT_FineTune(handle_encoder='/home/jupyter/BertPretained/bert_en_uncased_L-12_H-768_A-12', 
#                               handle_preprocess='/home/jupyter/BertPreprocess/bert_en_uncased_preprocess')

bert_finetune = BERT_FineTune(handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3', 
                              handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

Running on CPU is not recommended.


In [ ]:
bert_model = bert_finetune.tune(train_df, val_df, feature_cols,
                   label='sentiment', epochs=2,
                   batch_size=32, optimizer='sgd', seq_length=128)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['label'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Epoch 1/2
1250/1250 [==============================] - 2202s 2s/step - loss: 0.3608 - accuracy: 0.8356 - val_loss: 0.2752 - val_accuracy: 0.8818
Epoch 2/2
1250/1250 [==============================] - 2183s 2s/step - loss: 0.2664 - accuracy: 0.8881 - val_loss: 0.2787 - val_accuracy: 0.8802


In [12]:
bert_preprocess_model = bert_finetune.make_bert_preprocess_model(bert_finetune.handle_preprocess,feature_cols, seq_length=128)
test_ds, _=bert_finetune.convert_dataset(val_df, 32, 
                          bert_preprocess_model, 
                          feature_cols, 
                          label='sentiment', shuffle=False, repeat=False)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['label'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


In [ ]:
with bert_finetune.strategy.scope():
    predictions = bert_model.predict(test_ds)


In [ ]:
from sklearn.metrics import f1_score
import numpy as np
pred = np.argmax(predictions, axis=1)
real = val_df['sentiment']
f1_score(real, pred, average='macro')

0.8798348030624741

In [13]:
import tensorflow as tf

In [ ]:
tf.keras.models.save_model(
    bert_model, '/content/drive/MyDrive/Data/bert', overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True
)

NameError: ignored

In [ ]:
bert_model.save('/content/drive/MyDrive/Data/bert')

# load method 1

In [16]:
bert_model_loaded = tf.keras.models.load_model('/content/drive/MyDrive/Data/bert')

In [17]:
bert_model_loaded.predict(test_ds)

array([[-2.2573388 ,  2.1636329 ],
       [ 0.41836604, -0.2902224 ],
       [ 2.560231  , -3.053667  ],
       [ 2.8376184 , -2.3377025 ],
       [-1.938671  ,  1.6335995 ],
       [ 2.6738029 , -2.133901  ],
       [ 2.5080783 , -2.1009414 ],
       [ 2.6132226 , -2.2868807 ],
       [ 0.86485064, -0.79451114],
       [ 2.2084835 , -1.91831   ],
       [ 2.350346  , -2.1415944 ],
       [ 1.6419716 , -1.4620467 ],
       [ 1.5054134 , -1.2570333 ],
       [-1.4324028 ,  1.5337418 ],
       [ 3.1358187 , -2.735223  ],
       [-1.0985657 ,  1.2649761 ]], dtype=float32)

In [20]:
pred = np.argmax(bert_model_loaded.predict(test_ds), axis=1)

In [21]:
real = val_df['sentiment']

In [23]:
from sklearn.metrics import f1_score
import numpy as np
f1_score(real, pred, average='macro')

1.0

# Load method 2

In [24]:
bert_model_loaded = tf.saved_model.load(
    '/content/drive/MyDrive/Data/bert', tags=None, options=None
)


In [25]:
f = bert_model_loaded.signatures["serving_default"]

In [26]:
test_point = list(test_ds)[0][0]

In [27]:
pred = np.argmax(f(input_mask = test_point['input_mask'], input_type_ids = test_point['input_type_ids'], input_word_ids = test_point['input_word_ids'])['output_1'].numpy(), axis=1)

In [28]:
pred = np.argmax(f(**test_point)['output_1'].numpy(), axis=1)

In [29]:
from sklearn.metrics import f1_score
import numpy as np

batch_size = 32
real = val_df.head(batch_size)['sentiment']
f1_score(real, pred, average='macro')



1.0